In [4]:
# !pip install onnxruntime sacremoses onnxconverter-common onnxruntime-tools tf2onnx

     |████████████████████████████████| 4.9 MB 1.6 MB/s            
     |████████████████████████████████| 78 kB 53.1 MB/s            
     |████████████████████████████████| 212 kB 37.3 MB/s            
     |████████████████████████████████| 435 kB 25.8 MB/s            
     |████████████████████████████████| 46 kB 44.0 MB/s            
     |████████████████████████████████| 99 kB 60.0 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 55 kB 36.7 MB/s            
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22257 sha256=fdbb1e733ed555b135e31e9849e3097eab3f85c4de86b904cbf1e681492eeb45
  Stored in directory: /tmp/pip-ephem-wheel-cache-sckgbv6r/wheels/57/cb/6d/bab2257f26c5be4a96ff65c3d2a7122c96529b73773ee37f36
Successfully built py-cpuinfo
You should consider upgrading via the '/opt/conda/bin/python3 -m pip install --upgrade pip' command.


In [1]:
from transformers.onnx.features import FeaturesManager

distilbert_features = list(FeaturesManager.get_supported_features_for_model_type("bert").keys())

In [2]:
print(distilbert_features)

['default', 'masked-lm', 'causal-lm', 'sequence-classification', 'token-classification', 'question-answering']


In [14]:
!python -m transformers.onnx --help

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2022-03-15 15:03:00.695733: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
usage: Hugging Face ONNX Exporter tool [-h] -m MODEL
                                       [--feature {causal-lm,causal-lm-with-past,default,default-with-past,masked-lm,question-answering,seq2seq-lm,seq2seq-lm-with-past,sequence-classification,token-classification}]
                                       [--opset OPSET] [--atol ATOL]
                                       output

positional arguments:
  output                Path indicating where to store generated ONNX model.

optional arguments:
  -h, --help            sh

In [15]:
!git clone https://huggingface.co/airesearch/wangchanberta-base-att-spm-uncased

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Cloning into 'wangchanberta-base-att-spm-uncased'...
remote: Enumerating objects: 251, done.
remote: Counting objects: 100% (251/251), done.
remote: Compressing objects: 100% (249/249), done.
remote: Total 251 (delta 156), reused 0 (delta 0)00 KiB/s   
Receiving objects: 100% (251/251), 603.98 KiB | 138.00 KiB/s, done.
Resolving deltas: 100% (156/156), done.


In [16]:
cd wangchanberta-base-att-spm-uncased

/workspace/pythainlp-work/onnx/wangchanberta-base-att-spm-uncased


In [18]:
!git checkout finetuned@lst20-ner

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Branch 'finetuned@lst20-ner' set up to track remote branch 'finetuned@lst20-ner' from 'origin'.
Switched to a new branch 'finetuned@lst20-ner'


In [19]:
cd ..

/workspace/pythainlp-work/onnx


In [20]:
!mv wangchanberta-base-att-spm-uncased cat-model

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [1]:
!rm -rf onnx

In [6]:
!apt install -y git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 17 not upgraded.
Need to get 2129 kB of archives.
After this operation, 7662 kB of additional disk space will be used.
E: You don't have enough free space in /var/cache/apt/archives/.


In [7]:
!python -m transformers.onnx \
    --model=./cat-model \
    --feature=token-classification \
    onnx/

2022-03-15 15:23:41.907191: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Using framework PyTorch: 1.10.2+cu111
Overriding 1 configuration item(s)
	- use_cache -> False
/opt/conda/lib/python3.8/site-packages/torch/onnx/utils.py:90: UserWarning: 'enable_onnx_checker' is deprecated and ignored. It will be removed in the next PyTorch release. To proceed despite ONNX checker failures, catch torch.onnx.ONNXCheckerError.
  warnings.warn("'enable_onnx_checker' is deprecated and ignored. It will be removed in "
/opt/conda/lib/python3.8/site-packages/torch/onnx/utils.py:103: UserWarning: `use_external_data_format' is deprecated and ignored. Will be removed in next PyTorch release. The code will work as it is False if models are not larger than 2GB, Otherwise set to False because of size limits imposed by Protocol Buffers.
  warnings.warn("`use_external_data_format' is deprecated and ignored. Will be removed in next "
Valida

In [8]:
import torch

from onnxruntime import (
    InferenceSession, SessionOptions, GraphOptimizationLevel
)
from transformers import (
    TokenClassificationPipeline, AutoTokenizer, AutoModelForTokenClassification
)

2022-03-15 15:25:03.551592: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [9]:
options = SessionOptions() # initialize session options
options.graph_optimization_level = GraphOptimizationLevel.ORT_ENABLE_ALL

session = InferenceSession(
    "onnx/model.onnx", sess_options=options, providers=["CPUExecutionProvider"]
)

# disable session.run() fallback mechanism, it prevents for a reset of the execution provider
session.disable_fallback()

In [10]:
from onnxruntime import get_all_providers

get_all_providers()

['TensorrtExecutionProvider',
 'CUDAExecutionProvider',
 'MIGraphXExecutionProvider',
 'ROCMExecutionProvider',
 'OpenVINOExecutionProvider',
 'DnnlExecutionProvider',
 'NupharExecutionProvider',
 'VitisAIExecutionProvider',
 'NnapiExecutionProvider',
 'CoreMLExecutionProvider',
 'ArmNNExecutionProvider',
 'ACLExecutionProvider',
 'DmlExecutionProvider',
 'RknpuExecutionProvider',
 'CPUExecutionProvider']

In [15]:
class OnnxTokenClassificationPipeline(TokenClassificationPipeline):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    
    def _forward(self, model_inputs):
        """
        Forward pass through the model. This method is not to be called by the user directly and is only used
        by the pipeline to perform the actual predictions.
        This is where we will define the actual process to do inference with the ONNX model and the session created
        before.
        """

        # This comes from the original implementation of the pipeline
        special_tokens_mask = model_inputs.pop("special_tokens_mask")
        offset_mapping = model_inputs.pop("offset_mapping", None)
        sentence = model_inputs.pop("sentence")

        inputs = {k: v.cpu().detach().numpy() for k, v in model_inputs.items()} # dict of numpy arrays
        outputs_name = session.get_outputs()[0].name # get the name of the output tensor

        logits = session.run(output_names=[outputs_name], input_feed=inputs)[0] # run the session
        logits = torch.tensor(logits) # convert to torch tensor to be compatible with the original implementation

        return {
            "logits": logits,
            "special_tokens_mask": special_tokens_mask,
            "offset_mapping": offset_mapping,
            "sentence": sentence,
            **model_inputs,
        }

    # We need to override the preprocess method because the onnx model is waiting for the attention masks as inputs
    # along with the embeddings.
    def preprocess(self, sentence, offset_mapping=None):
        truncation = True if self.tokenizer.model_max_length and self.tokenizer.model_max_length > 0 else False
        model_inputs = self.tokenizer(
            sentence,
            return_attention_mask=True, # This is the only difference from the original implementation
            return_tensors=self.framework,
            truncation=truncation,
            return_special_tokens_mask=True,
            return_offsets_mapping=self.tokenizer.is_fast,
        )
        if offset_mapping:
            model_inputs["offset_mapping"] = offset_mapping

        model_inputs["sentence"] = sentence

        return model_inputs

In [49]:
model_name_from_hub = "./cat-model"

tokenizer = AutoTokenizer.from_pretrained(model_name_from_hub)
model = AutoModelForTokenClassification.from_pretrained(model_name_from_hub)

onnx_pipeline = OnnxTokenClassificationPipeline(
    task="ner", 
    model=model,
    tokenizer=tokenizer,
    framework="pt",
    aggregation_strategy="simple",
)

In [50]:
onnx_pipeline.framework

'pt'

In [51]:
model

CamembertForTokenClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(25005, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNo

In [52]:
sequence  = "ตอนนี้เวลา 19:18 น. เราจะไปไหน"

In [53]:
onnx_pipeline(sequence)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity_group': 'B_DTM',
  'score': 0.84454143,
  'word': 'เวลา 19:18',
  'start': 6,
  'end': 16},
 {'entity_group': 'E_DTM',
  'score': 0.98721534,
  'word': 'น.',
  'start': 16,
  'end': 19}]

In [54]:
len(sequence)

30

In [91]:
sequence2="ผมเป็นแมว"

In [98]:
tokenizer(sequence2)

{'input_ids': [5, 330, 17, 1624, 6], 'attention_mask': [1, 1, 1, 1, 1]}

In [106]:
c=tokenizer.get_vocab()

In [109]:
sorted([(i,c[i]) for i in c.keys()],key=lambda tup: tup[1]) 

[('<s>NOTUSED', 0),
 ('<pad>', 1),
 ('</s>NOTUSED', 2),
 ('<unk>', 3),
 ('<unk>NOTUSED', 4),
 ('<s>', 5),
 ('</s>', 6),
 ('<_>', 8),
 ('า', 9),
 ('▁', 10),
 ('.', 11),
 ('ที่', 12),
 ('และ', 13),
 (',', 14),
 ('ได้', 15),
 ('ของ', 16),
 ('เป็น', 17),
 ('"', 18),
 ('▁"', 19),
 ('ใน', 20),
 ('ให้', 21),
 ('แล้ว', 22),
 ('มี', 23),
 ('การ', 24),
 ('(', 25),
 ('มา', 26),
 ('ว่า', 27),
 ('ไป', 28),
 (')', 29),
 ('แต่', 30),
 ('หรือ', 31),
 ('จาก', 32),
 ('กับ', 33),
 ('ๆ', 34),
 ('-', 35),
 ('ก็', 36),
 ('จะ', 37),
 ('#', 38),
 ('คน', 39),
 ('ส', 40),
 ('ค', 41),
 ('น', 42),
 ('พ', 43),
 ('นี้', 44),
 ('ด้วย', 45),
 ('ต', 46),
 ('2', 47),
 ('เลย', 48),
 ('โดย', 49),
 ('อ', 50),
 ('ไม่', 51),
 ('คือ', 52),
 ('""', 53),
 ('เพื่อ', 54),
 ('อยู่', 55),
 ('ย', 56),
 ('?', 57),
 ('อย่าง', 58),
 ('1', 59),
 ('ซึ่ง', 60),
 ('ก', 61),
 (':', 62),
 ('ความ', 63),
 ('3', 64),
 ('5', 65),
 ('มัน', 66),
 ('กัน', 67),
 ('ปี', 68),
 ('เพราะ', 69),
 ('ค่ะ', 70),
 ('นาย', 71),
 ('นั้น', 72),
 ('ครับ', 73),
 

In [99]:
sp.encode(sequence2)

[326, 13, 1620]

In [55]:
model_inputs = tokenizer(
            sequence,
            return_attention_mask=True, # This is the only difference from the original implementation
            return_tensors='pt',
            truncation=True,
            return_special_tokens_mask=True,
            return_offsets_mapping=tokenizer.is_fast,
        )

In [56]:
model_inputs

{'input_ids': tensor([[    5,  1416,   156,    10,   439,    62,   522,    10,    42,    11,
           456, 10965,     6]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'special_tokens_mask': tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]), 'offset_mapping': tensor([[[ 0,  0],
         [ 0,  6],
         [ 6, 10],
         [10, 11],
         [11, 13],
         [13, 14],
         [14, 16],
         [16, 17],
         [17, 18],
         [18, 19],
         [19, 23],
         [23, 30],
         [ 0,  0]]])}

In [80]:
model_inputs['input_ids'][0]

tensor([    5,  1416,   156,    10,   439,    62,   522,    10,    42,    11,
          456, 10965,     6])

In [104]:
[i+4 for i in sp.encode(sequence)]

[1416, 156, 10, 439, 62, 522, 10, 42, 11, 456, 10965]

In [65]:
special_tokens_mask = model_inputs.pop("special_tokens_mask")
offset_mapping = model_inputs.pop("offset_mapping", None)
#sentence = model_inputs.pop("sentence")

In [66]:
inputs = {k: v.cpu().detach().numpy() for k, v in model_inputs.items()} 

In [117]:
inputs

{'input_ids': array([[    5,  1416,   156,    10,   439,    62,   522,    10,    42,
            11,   456, 10965,     6]]),
 'attention_mask': array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [67]:
outputs_name = session.get_outputs()[0].name

In [68]:
logits = session.run(output_names=[outputs_name], input_feed=inputs)[0]

In [69]:
logits = torch.tensor(logits)

In [89]:
logits[0][2]

tensor([ 5.3768, -2.0700, -0.5485,  8.1550, -0.4456, -0.2665, -0.9346, -0.1501,
        -0.7186, -0.9373, -0.8703, -1.9875, -1.0404,  3.8525, -0.9652, -0.2769,
        -2.3533,  0.4932, -1.5370, -1.8904, -1.7153, -2.6149, -1.1447,  1.0322,
        -2.3711, -1.1148, -2.1653, -0.1513, -0.9360, -2.0010, -1.8746])

In [77]:
len(logits[0][0])

31

In [71]:
import sentencepiece as spm

In [95]:
sp = spm.SentencePieceProcessor(model_file='cat-model/sentencepiece.bpe.model')

In [96]:
sequence

'ตอนนี้เวลา 19:18 น. เราจะไปไหน'

In [97]:
sp.EncodeAsPieces(sequence)

['▁ตอนนี้', 'เวลา', '▁', '19', ':', '18', '▁', 'น', '.', '▁เรา', 'จะไปไหน']

In [41]:
tokenizer

PreTrainedTokenizerFast(name_or_path='./cat-model', vocab_size=25005, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True), 'additional_special_tokens': ['<s>NOTUSED', '</s>NOTUSED', '<_>']})

In [111]:
import numpy as np

In [119]:
def build_tokenizer(sent):
    _t = [5]+[i+4 for i in sp.encode(sent)]+[6]
    model_inputs = {}
    model_inputs["input_ids"]=np.array([_t])
    model_inputs["attention_mask"]=np.array([[1]*len(_t)])
    return model_inputs

In [120]:
i_c=build_tokenizer(sequence)

In [121]:
i_c

{'input_ids': array([[    5,  1416,   156,    10,   439,    62,   522,    10,    42,
            11,   456, 10965,     6]]),
 'attention_mask': array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [125]:
logits = session.run(output_names=[outputs_name], input_feed=i_c)[0]

In [127]:
logits[0]

array([[ 4.380878  , -1.6673179 , -1.1982256 ,  6.3787613 , -1.0288123 ,
        -0.50097805, -0.47615245, -0.49537125, -0.49321952, -1.1285951 ,
        -0.7383426 , -1.1578354 , -2.0872838 ,  5.027358  , -1.4874482 ,
         0.23384863, -1.237948  , -0.7719777 , -1.0478671 , -1.7059261 ,
        -2.0042663 , -1.7156132 , -1.2701837 ,  4.7759585 , -1.2704831 ,
        -0.6026307 , -1.727892  , -0.780291  , -0.6802486 , -1.8957199 ,
        -2.2440815 ],
       [ 7.331868  , -1.5951151 , -0.9564124 ,  6.9455357 , -0.55916536,
        -0.28192207, -1.0282085 ,  0.11319103, -0.35786018, -1.6785506 ,
        -0.7617464 , -2.2966197 , -1.8616681 ,  2.5820844 , -0.7152224 ,
         0.3366683 , -1.6750131 ,  0.05212411, -1.5421431 , -2.1218092 ,
        -1.8217936 , -2.3663418 , -1.1259866 ,  0.8986639 , -1.4895829 ,
        -0.6464236 , -2.2468598 , -0.13723528, -1.196146  , -2.0824785 ,
        -1.7626154 ],
       [ 5.376783  , -2.0700448 , -0.54854214,  8.15497   , -0.4456482 ,
       

In [128]:
np.max(logits[0], axis=-1, keepdims=True)

array([[6.3787613],
       [7.331868 ],
       [8.15497  ],
       [8.420903 ],
       [8.453953 ],
       [8.258942 ],
       [7.6121593],
       [9.099442 ],
       [9.681296 ],
       [9.696796 ],
       [8.456515 ],
       [9.016193 ],
       [6.3787594]], dtype=float32)

In [129]:
def postprocess(logits_data):
    logits_t = logits_data[0]
    maxes = np.max(logits_t, axis=-1, keepdims=True)
    shifted_exp = np.exp(logits_t - maxes)
    scores = shifted_exp / shifted_exp.sum(axis=-1, keepdims=True)
    return scores

In [143]:
id2tag={
    "0": "O",
    "1": "B_BRN",
    "2": "B_DES",
    "3": "B_DTM",
    "4": "B_LOC",
    "5": "B_MEA",
    "6": "B_NUM",
    "7": "B_ORG",
    "8": "B_PER",
    "9": "B_TRM",
    "10": "B_TTL",
    "11": "I_BRN",
    "12": "I_DES",
    "13": "I_DTM",
    "14": "I_LOC",
    "15": "I_MEA",
    "16": "I_NUM",
    "17": "I_ORG",
    "18": "I_PER",
    "19": "I_TRM",
    "20": "I_TTL",
    "21": "E_BRN",
    "22": "E_DES",
    "23": "E_DTM",
    "24": "E_LOC",
    "25": "E_MEA",
    "26": "E_NUM",
    "27": "E_ORG",
    "28": "E_PER",
    "29": "E_TRM",
    "30": "E_TTL"
  }

In [160]:
def totag(post,sent):
    tag= []
    _s=sp.EncodeAsPieces(sent)
    for i in range(len(_s)):
        tag.append(
            (
                _s[i],
                id2tag[str(list(post[i+1]).index(max(list(post[i+1]))))]
            )
        )
    return tag
    

In [161]:
totag(postprocess(logits),sequence)

[('▁ตอนนี้', 'O'),
 ('เวลา', 'B_DTM'),
 ('▁', 'B_DTM'),
 ('19', 'B_DTM'),
 (':', 'B_DTM'),
 ('18', 'B_DTM'),
 ('▁', 'E_DTM'),
 ('น', 'E_DTM'),
 ('.', 'E_DTM'),
 ('▁เรา', 'O'),
 ('จะไปไหน', 'O')]

In [166]:
def ner(sent):
    _s=build_tokenizer(sent)
    logits = session.run(output_names=[outputs_name], input_feed=_s)[0]
    return totag(postprocess(logits),sent)

In [171]:
ner("เมื่อไรจะเวลา 11:00 น. เราจะไปทำอะไรที่โรเรียนอำเภอนางรอง จังหวัดบุรีรัมย์")

[('▁', 'O'),
 ('เมื่อไร', 'O'),
 ('จะ', 'O'),
 ('เวลา', 'B_DTM'),
 ('▁', 'B_DTM'),
 ('11', 'B_DTM'),
 (':00', 'B_DTM'),
 ('▁', 'E_DTM'),
 ('น', 'E_DTM'),
 ('.', 'E_DTM'),
 ('▁เราจะ', 'O'),
 ('ไปทําอะไร', 'O'),
 ('ที่', 'O'),
 ('โร', 'O'),
 ('เรียน', 'O'),
 ('อําเภอ', 'O'),
 ('นาง', 'O'),
 ('รอง', 'O'),
 ('▁', 'O'),
 ('จังหวัด', 'O'),
 ('บุรีรัมย์', 'O')]

In [154]:
list(postprocess(logits)[1]).index(list(postprocess(logits))[1].max())

0

In [141]:
max(list(postprocess(logits)[5]))

0.86657107

In [140]:
max(list(postprocess(logits)[6]))

0.6273591

In [138]:
list(postprocess(logits)[6])

[0.0021105963,
 5.2383824e-05,
 0.00013880829,
 0.6273591,
 6.666468e-05,
 0.00069932215,
 0.0006888598,
 7.789968e-05,
 7.235209e-05,
 0.00014373407,
 8.691182e-05,
 7.858766e-05,
 0.00011105001,
 0.35903227,
 9.880592e-05,
 0.00021588136,
 5.343647e-05,
 0.0002957834,
 9.390179e-05,
 5.8271926e-05,
 8.277429e-05,
 5.6007877e-05,
 5.2078285e-05,
 0.007708671,
 6.502185e-05,
 7.846872e-05,
 7.785043e-05,
 0.00012567629,
 0.00012432446,
 5.0244704e-05,
 4.43246e-05]

In [137]:
len(list(postprocess(logits)[0]))

31